In [5]:
import pandas as pd

off_train = pd.read_csv('data/ccf_offline_stage1_train.csv',parse_dates=["Date"],header=0)
off_train.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']

off_train.ix[:5]

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
0,1439408,2632,null,null,0,null,20160217
1,1439408,4663,11002,150:20,1,20160528,null
2,1439408,2632,8591,20:1,0,20160217,null
3,1439408,2632,1078,20:1,0,20160319,null
4,1439408,2632,8591,20:1,0,20160613,null
5,1439408,2632,null,null,0,null,20160516


In [6]:
off_test = pd.read_csv('data/ccf_offline_stage1_test_revised.csv',header=0)
off_test.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received']

off_test.ix[:5]

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received
0,4129537,450,9983,30:5,1,20160712
1,6949378,1300,3429,30:5,null,20160706
2,2166529,7113,6928,200:20,5,20160727
3,2166529,7113,1808,100:10,5,20160727
4,6172162,7605,6500,30:1,2,20160708
5,4005121,450,9983,30:5,0,20160706


In [7]:
on_train = pd.read_csv('data/ccf_online_stage1_train.csv',parse_dates=["Date"],header=0)
on_train.columns = ['user_id','merchant_id','action','coupon_id','discount_rate','date_received','date']

on_train.ix[:5]

,user_id,merchant_id,action,coupon_id,discount_rate,date_received,date
0,13740231,18907,2,100017492,500:50,20160513,null
1,13740231,34805,1,null,null,null,20160321
2,14336199,18907,0,null,null,null,20160618
3,14336199,18907,0,null,null,null,20160618
4,14336199,18907,0,null,null,null,20160618
5,14336199,18907,0,null,null,null,20160618


In [8]:
"""
 1、特征数据划分                            
                       预测区间                          特征区间
        【dataset3】20160701~20160731 (113640)   【feature3】20160315~20160630  （测试集）
        【dataset2】20160515~20160615 (258446)   【feature2】20160201~20160514  （训练集2）
        【dataset1】20160414~20160514 (138303)   【feature1】20160101~20160413  （训练集1）
"""

'\n 1、特征数据划分                            \n                       预测区间                          特征区间\n        【dataset3】20160701~20160731 (113640)   【feature3】20160315~20160630  （测试集）\n        【dataset2】20160515~20160615 (258446)   【feature2】20160201~20160514  （训练集2）\n        【dataset1】20160414~20160514 (138303)   【feature1】20160101~20160413  （训练集1）\n'

In [9]:
# 【dataset3】20160701~20160731 (113640)   【feature3】20160315~20160630  （测试集）
dataset3 = off_test.dropna(how='any')
feature3 = off_train[((off_train.date >= '20160315') & (off_train.date <= '20160630'))
                   | ((off_train.date == 'null') & (off_train.date_received >= '20160315') 
                      &  (off_train.date_received <= '20160630'))
                   ].dropna(how='any')
dataset3.ix[:5]
feature3.ix[:5]

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
1,1439408,4663,11002,150:20,1,20160528,null
3,1439408,2632,1078,20:1,0,20160319,null
4,1439408,2632,8591,20:1,0,20160613,null
5,1439408,2632,null,null,0,null,20160516


In [10]:
# 【dataset2】20160515~20160615 (258446)   【feature2】20160201~20160514  （训练集2）
dataset2 = off_train[((off_train.date_received >= '20160515') & (off_train.date_received <= '20160615'))].dropna(how='any')
feature2 = off_train[((off_train.date >= '20160201') & (off_train.date <= '20160514'))
                   | ((off_train.date == 'null') & (off_train.date_received >= '20160201') 
                      &  (off_train.date_received <= '20160514'))
                   ].dropna(how='any')
dataset2.ix[:5]
feature2.ix[:5]

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
0,1439408,2632,null,null,0,null,20160217
2,1439408,2632,8591,20:1,0,20160217,null
3,1439408,2632,1078,20:1,0,20160319,null


In [11]:
# 【dataset1】20160414~20160514 (138303)   【feature1】20160101~20160413  （训练集1）
dataset1 = off_train[((off_train.date_received >= '20160414') & (off_train.date_received <= '20160514'))].dropna(how='any')
feature1 = off_train[((off_train.date >= '20160101') & (off_train.date <= '20160413'))
                   | ((off_train.date == 'null') & (off_train.date_received >= '20160101') 
                      &  (off_train.date_received <= '20160413'))
                   ].dropna(how='any')
dataset1.ix[:5]
feature1.ix[:5]

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
0,1439408,2632,null,null,0,null,20160217
2,1439408,2632,8591,20:1,0,20160217,null
3,1439408,2632,1078,20:1,0,20160319,null


In [12]:
"""
    2、获得这8个特征，从训练集1、训练集2和测试集的预测区间提取： 
      this_month_user_receive_all_coupon_count 这个月用户收取的所有优惠券数目 
      this_month_user_receive_same_coupon_count 这个月用户收到的相同优惠券的数量 
      this_month_user_receive_same_coupon_lastone 这个月优惠券最远接受时间 
      this_month_user_receive_same_coupon_firstone 这个月优惠券最近接受时间 
      this_day_user_receive_all_coupon_count 一天内用户接收到所有优惠券的数量 
      this_day_user_receive_same_coupon_count 一天内用户接收到相同优惠券的数量 
      day_gap_before 用户上一次领取优惠券的时间间隔 
      day_gap_after  (receive the same coupon)用户下一次领取优惠券的时间间隔
"""

'\n    2、获得这8个特征，从训练集1、训练集2和测试集的预测区间提取： \n      this_month_user_receive_all_coupon_count 这个月用户收取的所有优惠券数目 \n      this_month_user_receive_same_coupon_count 这个月用户收到的相同优惠券的数量 \n      this_month_user_receive_same_coupon_lastone 这个月优惠券最远接受时间 \n      this_month_user_receive_same_coupon_firstone 这个月优惠券最近接受时间 \n      this_day_user_receive_all_coupon_count 一天内用户接收到所有优惠券的数量 \n      this_day_user_receive_same_coupon_count 一天内用户接收到相同优惠券的数量 \n      day_gap_before 用户上一次领取优惠券的时间间隔 \n      day_gap_after  (receive the same coupon)用户下一次领取优惠券的时间间隔\n'

In [13]:
from datetime import date
import numpy as np
def is_firstlastone(x):
    if x == 0:
         return 1
    elif x > 0:
        return 0
    else:
        return -1  # those only receive once

def get_day_gap_before(s):
    date_received, dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        af_date = date(int(date_received[0:4]), int(date_received[4:6]), int(date_received[6:8]))
        bf_date =  date(int(d[0:4]),int(d[4:6]),int(d[6:8]))
        this_gap = (af_date - bf_date).days
        if this_gap > 0:
            gaps.append(this_gap)
    if len(gaps) == 0:
        return -1
    else:
        return min(gaps)


def get_day_gap_after(s):
    date_received, dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        bf_date = date(int(date_received[0:4]), int(date_received[4:6]), int(date_received[6:8]))
        af_date =  date(int(d[0:4]),int(d[4:6]),int(d[6:8]))
        this_gap = (af_date - bf_date).days
        if this_gap > 0:
            gaps.append(this_gap)
    if len(gaps) == 0:
        return -1
    else:
        return min(gaps)

In [14]:
def dealOtherFeature(dateset3,file_name):
    #this_month_user_receive_all_coupon_count 这个月用户收取的所有优惠券数目 
    t = dataset3[['user_id']]
    t['this_month_user_receive_all_coupon_count'] = 1
    t = t.groupby('user_id').agg('sum').reset_index()
    t.ix[:5]
    #this_month_user_receive_same_coupon_count 这个月用户收到的相同优惠券的数量 
    t1 = dataset3[['user_id','coupon_id']]
    t1['this_month_user_receive_same_coupon_count'] = 1
    t1.groupby(['user_id','coupon_id']).agg('sum').reset_index()
    t1.ix[:5]
    #this_month_user_receive_same_coupon_lastone 这个月优惠券最远接受时间  
    #this_month_user_receive_same_coupon_firstone 这个月优惠券最近接受时间 
    t2 = dataset3[['user_id','coupon_id','date_received']]
    t2.date_received = t2.date_received.astype('str')
    t2 = t2.groupby(['user_id', 'coupon_id'])['date_received'].agg(lambda x: ':'.join(x)).reset_index()
    t2['receive_number'] = t2.date_received.apply(lambda s: len(s.split(':')))
    t2 = t2[t2.receive_number > 1]
    t2['max_date_received'] = t2.date_received.apply(lambda s: max([int(d) for d in s.split(':')]))
    t2['min_date_received'] = t2.date_received.apply(lambda s: min([int(d) for d in s.split(':')]))
    t2 = t2[['user_id','coupon_id','max_date_received','min_date_received']]
    t2.ix[:5]
    
    t3 = dataset3[['user_id','coupon_id','date_received']]
    t3 = pd.merge(t3,t2,on=['user_id','coupon_id'],how='left').dropna(how='any')
    t3.date_received = t3.date_received.astype('float')
    t3['this_month_user_receive_same_coupon_lastone'] = t3.max_date_received - t3.date_received
    t3['this_month_user_receive_same_coupon_firstone'] = t3.date_received - t3.min_date_received
    t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(
            is_firstlastone)
    t3.this_month_user_receive_same_coupon_firstone = t3.this_month_user_receive_same_coupon_firstone.apply(
            is_firstlastone)
    t3 = t3[['user_id', 'coupon_id', 'date_received', 'this_month_user_receive_same_coupon_lastone',
                 'this_month_user_receive_same_coupon_firstone']]
    t3.ix[:5]
    
    #this_day_user_receive_all_coupon_count 一天内用户接收到所有优惠券的数量  
    t4 = dataset3[['user_id','date_received']]
    t4['this_day_user_receive_all_coupon_count'] = 1
    t4 = t4.groupby(['user_id','date_received']).agg('sum').reset_index()
    t4.ix[:5]
    
    #this_day_user_receive_same_coupon_count 一天内用户接收到相同优惠券的数量
    t5 = dataset3[['user_id','coupon_id','date_received']]
    t5['this_day_user_receive_same_coupon_count'] = 1
    t5 = t5.groupby(['user_id','coupon_id','date_received']).agg('sum').reset_index()
    t5.ix[:5]
    
    #day_gap_after  用户下一次领取优惠券的时间间隔 (同一优惠券)
    #day_gap_before 用户上一次领取优惠券的时间间隔 
    t6 = dataset3[['user_id','coupon_id','date_received']]
    t6.date_received = t6.date_received.astype('str')
    t6 = t6.groupby(['user_id', 'coupon_id'])['date_received'].agg(lambda x: ':'.join(x)).reset_index()
    t6.rename(columns={'date_received': 'dates'}, inplace=True)
     
    t7 = dataset3[['user_id','coupon_id','date_received']]
    t7 = pd.merge(t7,t6,on=['user_id','coupon_id'],how='left').dropna(how='any')
    t7['date_received_date'] = t7.date_received.astype('str') + '-' + t7.dates

    t7['day_gap_before'] = t7.date_received_date.apply(get_day_gap_before)
    t7['day_gap_after'] = t7.date_received_date.apply(get_day_gap_after)
    t7 = t7[['user_id', 'coupon_id', 'date_received', 'day_gap_before', 'day_gap_after']]
    t7.ix[:5]
    
    # 合并所有的特征
    other_feature3 = pd.merge(t1,t,on='user_id').dropna(how='any')
    other_feature3 = pd.merge(other_feature3,t3,on=['user_id','coupon_id']).dropna(how='any')
    other_feature3 = pd.merge(other_feature3,t4,on=['user_id','date_received']).dropna(how='any')
    other_feature3 = pd.merge(other_feature3,t5,on=['user_id','coupon_id','date_received']).dropna(how='any')
    other_feature3 = pd.merge(other_feature3,t7,on=['user_id','coupon_id','date_received']).dropna(how='any')

    other_feature3.to_csv(file_name)
    return other_feature3

In [15]:
other_feature3 = dealOtherFeature(dataset3,'data/other_feature3.csv')
other_feature3.ix[:5]

C:\Users\xhb_1\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\xhb_1\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\xhb_1\Anaconda3\lib\site-packages\pandas\core\generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

,user_id,coupon_id,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,date_received,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after
0,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1
1,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1
2,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1
3,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1
4,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1
5,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1


In [16]:
other_feature2 = dealOtherFeature(dataset2,'data/other_feature2.csv')
other_feature2.ix[:5]

C:\Users\xhb_1\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\xhb_1\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\xhb_1\Anaconda3\lib\site-packages\pandas\core\generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

,user_id,coupon_id,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,date_received,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after
0,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1
1,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1
2,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1
3,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1
4,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1
5,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1


In [17]:
other_feature1 = dealOtherFeature(dataset1,'data/other_feature1.csv')
other_feature1.ix[:5]

C:\Users\xhb_1\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\xhb_1\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\xhb_1\Anaconda3\lib\site-packages\pandas\core\generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

,user_id,coupon_id,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,date_received,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after
0,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1
1,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1
2,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1
3,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1
4,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1
5,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1


In [18]:
"""
3、商户相关特征，9个，从训练集1、训练集2和测试集的特征区间提取： 
        total_sales # 每个商户的销售数量
        sales_use_coupon # 使用了优惠券消费的商品
        total_coupon # 商品的优惠券的总数量

        merchant_mean_distance # 所有使用优惠券消费的用户与商户的距离平均值
        merchant_median_distance # 所有使用优惠券消费的用户与商户的距离中位值
        merchant_min_distance # 所有使用优惠券消费的用户与商户的距离最小值
        merchant_max_distance # 所有使用优惠券消费的用户与商户的距离最大值
        
        coupon_rate = sales_use_coupon/total_sales # 卖出商品中使用优惠券的占比
        merchant_coupon_transfer_rate = sales_use_coupon/total_coupon # 优惠券的使用率

"""


'\n3、商户相关特征，9个，从训练集1、训练集2和测试集的特征区间提取： \n        total_sales # 每个商户的销售数量\n        sales_use_coupon # 使用了优惠券消费的商品\n        total_coupon # 商品的优惠券的总数量\n\n        merchant_mean_distance # 所有使用优惠券消费的用户与商户的距离平均值\n        merchant_median_distance # 所有使用优惠券消费的用户与商户的距离中位值\n        merchant_min_distance # 所有使用优惠券消费的用户与商户的距离最小值\n        merchant_max_distance # 所有使用优惠券消费的用户与商户的距离最大值\n        \n        coupon_rate = sales_use_coupon/total_sales # 卖出商品中使用优惠券的占比\n        merchant_coupon_transfer_rate = sales_use_coupon/total_coupon # 优惠券的使用率\n\n'

In [19]:
def merchantRelatedFeature(feature3 ,file_name):
    merchant3 = feature3[['merchant_id', 'coupon_id', 'distance', 'date_received', 'date']]
    merchant3.ix[:5]
    
    #total_sales # 显示每个商户的销售数量
    t = merchant3[['merchant_id']]
    t.drop_duplicates(inplace=True)

    t1 = merchant3[merchant3.date != 'null'][['merchant_id']]
    t1['total_sales'] = 1
    t1 = t1.groupby('merchant_id').agg('sum').reset_index()
    t1.ix[:5]
    
    #sales_use_coupon # 显示使用了优惠券消费的商品
    t2 = merchant3[(merchant3.date != 'null') & (merchant3.coupon_id != 'null')][['merchant_id']]
    t2['sales_use_coupon'] = 1
    t2 = t2.groupby('merchant_id').agg('sum').reset_index()
    t2.ix[:5]
    
    # total_coupon # 商品的优惠券的总数量
    t3 = merchant3[merchant3.coupon_id != 'null'][['merchant_id']]
    t3['total_coupon'] = 1
    t3 = t3.groupby('merchant_id').agg('sum').reset_index()
    t3.ix[:5]

    t4 = merchant3[(merchant3.date != 'null') & (merchant3.coupon_id != 'null')][['merchant_id', 'distance']]
    t4.replace('null', -1, inplace=True)
    t4.distance = t4.distance.astype('int')
    t4.replace(-1, np.nan, inplace=True)
    t4.ix[:5]
    # merchant_min_distance # 所有使用优惠券消费的用户与商户的距离最小值
    t5 = t4.groupby('merchant_id').agg('min').reset_index()
    t5.rename(columns={'distance': 'merchant_min_distance'}, inplace=True)
    t5.ix[:5]
    # merchant_max_distance # 所有使用优惠券消费的用户与商户的距离最大值
    t6 = t4.groupby('merchant_id').agg('max').reset_index()
    t6.rename(columns={'distance': 'merchant_max_distance'}, inplace=True)
    t6.ix[:5]
    # merchant_mean_distance # 所有使用优惠券消费的用户与商户的距离平均值
    t7 = t4.groupby('merchant_id').agg('mean').reset_index()
    t7.rename(columns={'distance': 'merchant_mean_distance'}, inplace=True)
    t7.ix[:5]
    #merchant_median_distance # 所有使用优惠券消费的用户与商户的距离中位值
    t8 = t4.groupby('merchant_id').agg('median').reset_index()
    t8.rename(columns={'distance': 'merchant_median_distance'}, inplace=True)
    t8.ix[:5]
    
    # 合并以上特征数据
    merchant3_feature = pd.merge(t, t1, on='merchant_id', how='left')
    merchant3_feature = pd.merge(merchant3_feature, t2, on='merchant_id', how='left')
    merchant3_feature = pd.merge(merchant3_feature, t3, on='merchant_id', how='left')
    merchant3_feature = pd.merge(merchant3_feature, t5, on='merchant_id', how='left')
    merchant3_feature = pd.merge(merchant3_feature, t6, on='merchant_id', how='left')
    merchant3_feature = pd.merge(merchant3_feature, t7, on='merchant_id', how='left')
    merchant3_feature = pd.merge(merchant3_feature, t8, on='merchant_id', how='left')
    
    merchant3_feature = merchant3_feature.replace(np.nan, 0)  # fillna with 0
    merchant3_feature.ix[:5]
    
    # coupon_rate = sales_use_coupon/total_sales # 卖出商品中使用优惠券的占比
    merchant3_feature['coupon_rate'] = merchant3_feature.sales_use_coupon/merchant3_feature.total_sales
    merchant3_feature.ix[:5]
    # merchant_coupon_transfer_rate = sales_use_coupon/total_coupon # 优惠券的使用率
    merchant3_feature['merchant_coupon_transfer_rate'] = merchant3_feature.sales_use_coupon/merchant3_feature.total_coupon
    merchant3_feature.ix[:5]
    merchant3_feature.to_csv(file_name, index=None)
    return merchant3_feature

In [20]:
merchant3_feature = merchantRelatedFeature(feature3 ,'data/merchant3_feature.csv')
merchant3_feature.ix[:5]

C:\Users\xhb_1\Anaconda3\lib\site-packages\pandas\util\decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


,merchant_id,total_sales,sales_use_coupon,total_coupon,merchant_min_distance,merchant_max_distance,merchant_mean_distance,merchant_median_distance,coupon_rate,merchant_coupon_transfer_rate
0,4663,393.0,39.0,11445.0,0.0,10.0,1.911765,1.0,0.099237,0.003408
1,2632,8.0,3.0,23.0,0.0,1.0,0.666667,1.0,0.375000,0.130435
2,3381,9579.0,416.0,46729.0,0.0,10.0,1.713580,1.0,0.043428,0.008902
3,450,7757.0,1239.0,35933.0,0.0,10.0,0.894068,0.0,0.159727,0.034481
4,6459,20.0,0.0,16.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
5,6901,8939.0,994.0,13886.0,0.0,10.0,0.557895,0.0,0.111198,0.071583


In [21]:
merchant2_feature = merchantRelatedFeature(feature2 ,'data/merchant2_feature.csv')
merchant2_feature.ix[:5]

C:\Users\xhb_1\Anaconda3\lib\site-packages\pandas\util\decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


,merchant_id,total_sales,sales_use_coupon,total_coupon,merchant_min_distance,merchant_max_distance,merchant_mean_distance,merchant_median_distance,coupon_rate,merchant_coupon_transfer_rate
0,2632,14.0,3.0,31.0,1.0,1.0,1.000000,1.0,0.214286,0.096774
1,3381,11537.0,1980.0,56499.0,0.0,10.0,1.697002,1.0,0.171622,0.035045
2,2099,5711.0,1497.0,12215.0,0.0,10.0,1.004814,0.0,0.262126,0.122554
3,1569,493.0,91.0,25173.0,0.0,10.0,2.388235,1.0,0.184584,0.003615
4,8390,1207.0,133.0,690.0,0.0,10.0,0.865079,0.0,0.110191,0.192754
5,7884,1541.0,184.0,2686.0,0.0,10.0,1.180124,0.0,0.119403,0.068503


In [22]:
merchant1_feature = merchantRelatedFeature(feature1 ,'data/merchant1_feature.csv')
merchant1_feature.ix[:5]

C:\Users\xhb_1\Anaconda3\lib\site-packages\pandas\util\decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


,merchant_id,total_sales,sales_use_coupon,total_coupon,merchant_min_distance,merchant_max_distance,merchant_mean_distance,merchant_median_distance,coupon_rate,merchant_coupon_transfer_rate
0,2632,14.0,1.0,28.0,1.0,1.0,1.000000,1.0,0.071429,0.035714
1,3381,14962.0,2168.0,100369.0,0.0,10.0,1.650457,1.0,0.144900,0.021600
2,2099,5783.0,1705.0,16824.0,0.0,10.0,0.968072,0.0,0.294830,0.101343
3,4833,522.0,116.0,8321.0,0.0,10.0,3.037736,2.0,0.222222,0.013941
4,8390,1056.0,133.0,690.0,0.0,10.0,0.865079,0.0,0.125947,0.192754
5,1041,2078.0,308.0,8058.0,0.0,10.0,1.052265,0.0,0.148219,0.038223


In [23]:
"""
4、优惠券相关特征，8个，从训练集1、训练集2和测试集的预测区间提取：

        day_of_week # 显示时间是第几周
        day_of_month # 显示时间是几月
        days_distance# 优惠券领取日期和截止日之间的间隔天数
        is_weekend //优惠券领取日期是否属于周末
        
        discount_rate # 优惠券折扣率
        discount_man # 显示满了多少钱后开始减
        discount_jian # 显示满减的减少的钱
        is_man_jian # 返回优惠券是否是满减券
        
"""

'\n4、优惠券相关特征，8个，从训练集1、训练集2和测试集的预测区间提取：\n\n        day_of_week # 显示时间是第几周\n        day_of_month # 显示时间是几月\n        days_distance# 优惠券领取日期和截止日之间的间隔天数\n        is_weekend //优惠券领取日期是否属于周末\n        \n        discount_rate # 优惠券折扣率\n        discount_man # 显示满了多少钱后开始减\n        discount_jian # 显示满减的减少的钱\n        is_man_jian # 返回优惠券是否是满减券\n        \n'

In [24]:
def get_discount_man(s):
    s = s.split(':')
    if len(s) == 1:
        return 'null'
    else:
        return s[0]
def get_discount_jian(s):
    s = s.split(':')
    if len(s) == 1:
        return 'null'
    else:
        return s[1]
def is_man_jian(s):
    s = s.split(':')
    return len(s) != 1
def calc_discount_rate(s):
    s = s.split(':')
    if len(s) == 1: #直接折扣
        return float(s[0])
    else: #满X减Y
        return 1-float(s[1])/float(s[0])

In [25]:
def couponRelationFeature(dataset3 , file_name):
    #day_of_week # 显示时间是第几周
    dataset3['day_of_week'] = dataset3.date_received.astype('str').apply(
            lambda x: date(int(x[0:4]), int(x[4:6]), int(x[6:8])).weekday() + 1)
    dataset3.ix[:5]
    #day_of_month # 显示时间是几月
    dataset3['day_of_month'] = dataset3.date_received.astype('str').apply(
            lambda x: int(x[4:6]))
    dataset3.ix[:5]
    #days_distance# 优惠券领取日期和截止日之间的间隔天数
    dataset3['days_distance'] = dataset3.date_received.astype('str').apply(
            lambda x: (date(int(x[0:4]), int(x[4:6]), int(x[6:8])) - date(2016, 6, 30)).days)
    dataset3.ix[:5]
    #is_weekend //优惠券领取日期是否属于周末
    dataset3['is_weekend'] = dataset3.day_of_month.apply(
            lambda x: x == 6 | x == 7)
    dataset3.ix[:5]
    # discount_man # 显示满了多少钱后开始减
    dataset3['discount_man'] = dataset3.discount_rate.apply(get_discount_man)
    dataset3.ix[:5]
    # discount_jian # 显示满减的减少的钱
    dataset3['discount_jian'] = dataset3.discount_rate.apply(get_discount_jian)
    dataset3.ix[:5]
    #  is_man_jian # 返回优惠券是否是满减券
    dataset3['is_man_jian'] = dataset3.discount_rate.apply(is_man_jian)
    dataset3.ix[:5]
    
    d = dataset3
    # discount_rate # 优惠券折扣率
    d['discount_rate'] = d.discount_rate.apply(calc_discount_rate)
    d.ix[:5]
    
    d['coupon_count'] = 1
    d = d.groupby('coupon_id').agg('sum').reset_index()
    d.to_csv(file_name, index=None)
    return d

In [26]:
coupon3_feature = couponRelationFeature(dataset3 , 'data/coupon3_feature.csv')
coupon3_feature.ix[:5]

,coupon_id,user_id,merchant_id,discount_rate,date_received,day_of_week,day_of_month,days_distance,is_weekend,is_man_jian,coupon_count
0,3,48778380,81200,15.68,322571390,79,112,190,16.0,16.0,16
1,7,2667831,672,0.90,20160714,4,7,14,1.0,1.0,1
2,8,366099213,196134,87.30,1955589803,443,679,1903,97.0,97.0,97
3,14,571850657,466610,130.50,2923304708,645,1015,3208,145.0,145.0,145
4,20,71591067,61608,19.20,483857293,113,168,493,24.0,24.0,24
5,29,29277553,61327,6.65,141125038,40,49,138,7.0,7.0,7


In [27]:
coupon2_feature = couponRelationFeature(dataset2 , 'data/coupon2_feature.csv')
coupon2_feature.ix[:5]

,coupon_id,user_id,merchant_id,discount_rate,day_of_week,day_of_month,days_distance,is_weekend,is_man_jian,coupon_count
0,1,11351131,27556,3.80,14,22,-121,0.0,4.0,4
1,10,87513732,207930,27.55,131,145,-1224,0.0,29.0,29
2,100,24585855,48060,5.40,36,31,-198,0.0,6.0,6
3,10001,3434009,16460,1.90,4,11,-53,0.0,0.0,2
4,10002,79420274,34992,16.20,83,98,-528,0.0,18.0,18
5,10004,72281005,122136,18.00,101,120,-919,0.0,24.0,24


In [28]:
coupon1_feature = couponRelationFeature(dataset1 , 'data/coupon1_feature.csv')
coupon1_feature.ix[:5]

,coupon_id,user_id,merchant_id,discount_rate,day_of_week,day_of_month,days_distance,is_weekend,is_man_jian,coupon_count
0,1,639303,6889,0.95,5,5,-48,0.0,1.0,1
1,10,5681700,28680,3.80,21,20,-191,0.0,4.0,4
2,100,2882918,8010,0.90,6,5,-47,0.0,1.0,1
3,10005,9553670,3858,1.80,6,10,-114,0.0,2.0,2
4,10006,24675915,35301,6.65,30,32,-418,0.0,7.0,7
5,10009,18840903,42606,7.50,31,36,-621,0.0,9.0,9


In [29]:
"""
5、用户相关特征，14个，从训练集1、训练集2和测试集的特征区间提取：
        count_merchant # 用户消费商户数量
        
        user_mean_distance # 所有使用优惠券消费的商户与用户的平均距离
        user_min_distance # 所有使用优惠券消费的商户与用户的最小距离
        user_max_distance # 所有使用优惠券消费的商户与用户的最大距离
        user_median_distance # 所有使用优惠券消费的商户与用户的中位距离
        
        coupon_received # 用户领取优惠券次数
        buy_total # 用户消费次数
        buy_use_coupon # 每个用户使用优惠券消费次数
        avg_user_date_datereceived_gap # 用户从领取优惠券到消费的平均时间间隔
        min_user_date_datereceived_gap # 用户从领取优惠券到消费的最小时间间隔
        max_user_date_datereceived_gap # 用户从领取优惠券到消费的最大时间间隔
        user_coupon_transfer_rate = buy_use_coupon/coupon_received # 用户优惠券转化为实际消费比例
        buy_use_coupon_rate = buy_use_coupon/buy_total # 用户使用优惠券消费占总消费的比例
        user_date_datereceived_gap # 接受到优惠券的日期和使用之间的间隔


"""

'\n5、用户相关特征，14个，从训练集1、训练集2和测试集的特征区间提取：\n        count_merchant # 用户消费商户数量\n        \n        user_avg_distance # 所有使用优惠券消费的商户与用户的平均距离\n        user_min_distance # 所有使用优惠券消费的商户与用户的最小距离\n        user_max_distance # 所有使用优惠券消费的商户与用户的最大距离\n        user_median_distance # 所有使用优惠券消费的商户与用户的中位距离\n        \n        coupon_received # 用户领取优惠券次数\n        buy_total # 用户消费次数\n        buy_use_coupon # 每个用户使用优惠券消费次数\n        avg_user_date_datereceived_gap # 用户从领取优惠券到消费的平均时间间隔\n        min_user_date_datereceived_gap # 用户从领取优惠券到消费的最小时间间隔\n        max_user_date_datereceived_gap # 用户从领取优惠券到消费的最大时间间隔\n        user_coupon_transfer_rate = buy_use_coupon/coupon_received # 用户优惠券转化为实际消费比例\n        buy_use_coupon_rate = buy_use_coupon/buy_total # 用户使用优惠券消费占总消费的比例\n        user_date_datereceived_gap # 接受到优惠券的日期和使用之间的间隔\n\n\n'

In [67]:
user3 = feature3[['user_id', 'merchant_id', 'coupon_id', 'discount_rate', 'distance', 'date_received', 'date']]
user3.ix[:5]

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
1,1439408,4663,11002,150:20,1,20160528,null
3,1439408,2632,1078,20:1,0,20160319,null
4,1439408,2632,8591,20:1,0,20160613,null
5,1439408,2632,null,null,0,null,20160516


In [68]:
t = user3[['user_id']]
t.drop_duplicates(inplace=True)

C:\Users\xhb_1\Anaconda3\lib\site-packages\pandas\util\decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


In [69]:
# count_merchant # 用户消费商户数量
t1 = user3[user3.date != 'null'][['user_id','merchant_id']]
t1['count_merchant'] = 1
t1 = t1.groupby('user_id').agg('sum').reset_index()
t1.ix[:5]

,user_id,merchant_id,count_merchant
0,165,42774,12
1,209,3267,1
2,239,3465,1
3,285,450,1
4,315,8537,3
5,316,14616,6


In [70]:
t2 = user3[(user3.date != 'null') & (user3.coupon_id != 'null')][['user_id', 'distance']]
t2.replace('null', 0, inplace=True)
t2.distance = t2.distance.astype('int')
t2.replace(-1, np.nan, inplace=True)
t2.ix[:5]

,user_id,distance


In [71]:
# user_mean_distance # 所有使用优惠券消费的商户与用户的平均距离
t3 = t2.groupby('user_id').agg('mean').reset_index()
t3.rename(columns={'distance': 'user_mean_distance'}, inplace=True)
t3.ix[:5]

,user_id,user_mean_distance
0,417,0.0
1,687,0.0
2,696,0.0
3,947,7.0
4,1302,0.0
5,1318,0.0


In [72]:
# user_min_distance # 所有使用优惠券消费的商户与用户的最小距离
t4 = t2.groupby('user_id').agg('min').reset_index()
t4.rename(columns={'distance': 'user_min_distance'}, inplace=True)
t4.ix[:5]

,user_id,user_min_distance
0,417,0
1,687,0
2,696,0
3,947,7
4,1302,0
5,1318,0


In [73]:
# user_max_distance # 所有使用优惠券消费的商户与用户的最大距离
t5 = t2.groupby('user_id').agg('max').reset_index()
t5.rename(columns={'distance': 'user_max_distance'}, inplace=True)
t5.ix[:5]

,user_id,user_max_distance
0,417,0
1,687,0
2,696,0
3,947,7
4,1302,0
5,1318,0


In [74]:
# user_median_distance # 所有使用优惠券消费的商户与用户的中位距离
t6 = t2.groupby('user_id').agg('median').reset_index()
t6.rename(columns={'distance': 'user_median_distance'}, inplace=True)
t6.ix[:5]

,user_id,user_median_distance
0,417,0.0
1,687,0.0
2,696,0.0
3,947,7.0
4,1302,0.0
5,1318,0.0


In [75]:
# coupon_received # 用户领取优惠券次数
t7 = user3[user3.coupon_id != 'null'][['user_id']]
t7['coupon_received'] = 1
t7 = t7.groupby('user_id').agg('sum').reset_index()
t7.ix[:5]

,user_id,coupon_received
0,4,1
1,165,2
2,166,1
3,173,1
4,215,1
5,236,1


In [76]:
# buy_total # 用户消费次数
t8 = user3[(user3.date != 'null')][['user_id']]
t8['buy_total'] = 1
t8 = t8.groupby('user_id').agg('sum').reset_index()
t8.ix[:5]

,user_id,buy_total
0,165,12
1,209,1
2,239,1
3,285,1
4,315,3
5,316,6


In [77]:
# buy_use_coupon # 每个用户使用优惠券消费次数
t9 = user3[(user3.coupon_id != 'null')&(user3.date != 'null')][['user_id']]
t9['buy_total'] = 1
t9 = t9.groupby('user_id').agg('sum').reset_index()
t9.ix[:5]

,user_id,buy_total
0,417,1
1,687,1
2,696,4
3,947,1
4,1302,1
5,1318,2


In [78]:
def get_user_date_datereceived_gap(s):
    s = s.split(':')
    used = s[0]
    received = s[1]
    return ((date(int(used[0:4]),int(used[4:6]),int(used[6:8]))) - 
            (date(int(received[0:4]),int(received[4:6]),int(received[6:8])))).days
t10 = user3[(user3.date != 'null')&(user3.date_received != 'null')][['user_id','date_received','date']]
t10['user_date_datereceived_gap'] = t10.date + ":"+ t10.date_received
t10.user_date_datereceived_gap = t10.user_date_datereceived_gap.apply(get_user_date_datereceived_gap)
t10.ix[:5]

,user_id,date_received,date,user_date_datereceived_gap


In [79]:
# avg_user_date_datereceived_gap # 用户从领取优惠券到消费的平均时间间隔
t11 = t10.groupby('user_id').agg('mean').reset_index()
t11.rename(columns={'user_date_datereceived_gap': 'avg_user_date_datereceived_gap'}, inplace=True)
t11.ix[:5]

,user_id,avg_user_date_datereceived_gap
0,417,14.0
1,687,5.0
2,696,3.0
3,947,1.0
4,1302,10.0
5,1318,5.0


In [80]:
# min_user_date_datereceived_gap # 用户从领取优惠券到消费的最小时间间隔
t12 = t10.groupby('user_id').agg('min').reset_index()
t12.rename(columns={'user_date_datereceived_gap': 'min_user_date_datereceived_gap'}, inplace=True)
t12.ix[:5]

,user_id,date_received,date,min_user_date_datereceived_gap
0,417,20160329,20160412,14
1,687,20160328,20160402,5
2,696,20160411,20160413,1
3,947,20160609,20160610,1
4,1302,20160525,20160604,10
5,1318,20160505,20160506,1


In [81]:
# max_user_date_datereceived_gap # 用户从领取优惠券到消费的最大时间间隔
t13 = t10.groupby('user_id').agg('max').reset_index()
t13.rename(columns={'user_date_datereceived_gap': 'max_user_date_datereceived_gap'}, inplace=True)
t13.ix[:5]

,user_id,date_received,date,max_user_date_datereceived_gap
0,417,20160329,20160412,14
1,687,20160328,20160402,5
2,696,20160524,20160530,6
3,947,20160609,20160610,1
4,1302,20160525,20160604,10
5,1318,20160518,20160527,9


In [86]:
user3_feature = pd.merge(t, t1, on='user_id', how='left')
user3_feature = pd.merge(user3_feature, t3, on='user_id', how='left')
user3_feature = pd.merge(user3_feature, t4, on='user_id', how='left')
user3_feature = pd.merge(user3_feature, t5, on='user_id', how='left')
user3_feature = pd.merge(user3_feature, t6, on='user_id', how='left')
user3_feature = pd.merge(user3_feature, t7, on='user_id', how='left')
user3_feature = pd.merge(user3_feature, t8, on='user_id', how='left')
user3_feature = pd.merge(user3_feature, t9, on='user_id', how='left')
user3_feature = pd.merge(user3_feature, t11, on='user_id', how='left')
user3_feature = pd.merge(user3_feature, t12, on='user_id', how='left')
user3_feature = pd.merge(user3_feature, t13, on='user_id', how='left')

user3_feature = user3_feature.replace(np.nan, 0)
user3_feature.ix[:5]

,user_id,merchant_id,count_merchant,user_mean_distance,user_min_distance,user_max_distance,user_median_distance,coupon_received,buy_total_x,buy_total_y,avg_user_date_datereceived_gap,date_received_x,date_x,min_user_date_datereceived_gap,date_received_y,date_y,max_user_date_datereceived_gap
0,1439408,5264.0,2.0,NaN,NaN,NaN,NaN,4.0,2.0,1.0,28.0,20160516,20160613,28.0,20160516,20160613,28.0
1,1832624,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2029232,12918.0,2.0,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2747744,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,196342,1579.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,163606,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
# user_coupon_transfer_rate = buy_use_coupon/coupon_received # 用户优惠券转化为实际消费比例
# buy_use_coupon_rate = buy_use_coupon/buy_total # 用户使用优惠券消费占总消费的比例
# user_date_datereceived_gap # 接受到优惠券的日期和使用之间的间隔